![Immoscout](https://raw.githubusercontent.com/juliandnl/redi_ss20/master/image.png)

# Project - Cleaning the Expanded Immobilien Scout 24 Dataset

--- 

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px

In [4]:
# Read in dataset
df = pd.read_pickle("/content/berlin_housing_with_scraped_class_cleaned.pkl")

In [5]:
df.head()

,url,region,condition,rooms,rent,year_construction,space,cold_rent,utilities,heating_costs,security_deposit,parking_space,renovated_date,property_condition,furnishing,heating_type,energy_sources,energy_certificate,type_of_certificate,energy_requirement,efficiency_class,deposit_cleaned,n_missing
0,https://www.immobilienscout24.de/expose/116051687,Mitte,first_time_use,4.0,2659.00,2019,117.20,2199.000000,+460€,in,3,120€,NaN,New,NaN,Underfloor Heating,NaN,NaN,Bedarfsausweis,NaN,B,7977.00,5
1,https://www.immobilienscout24.de/expose/115338103,Kreuzberg,first_time_use,1.0,1200.00,2020,29.33,973.510000,"+226,49€",in,2920.53,NaN,NaN,New,Gehobene Qualität,Central Heating,Fernwärme Energieausweis liegt vor,liegt vor,NaN,NaN,NaN,2920.53,5
2,https://www.immobilienscout24.de/expose/116458710,Köpenick,well_kept,2.0,979.00,1997,83.61,799.000000,+90€,+90€,3,NaN,NaN,Groomed,Normale Qualität,Central Heating,Fernwärme Energieausweis liegt vor,liegt vor,Verbrauchsausweis Energie­ver­brauchs­kennwert...,NaN,NaN,2937.00,4
3,https://www.immobilienscout24.de/expose/116573177,Wilmersdorf,well_kept,4.0,1830.22,1900,171.18,911.544216,NaN,NaN,nan,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5490.66,13
4,https://www.immobilienscout24.de/expose/115925878,Kreuzberg,first_time_use,2.0,2272.00,2020,88.27,1131.573505,NaN,NaN,nan,NaN,NaN,New,Gehobene Qualität,Central Heating,Fernwärme Energieausweis liegt zur Besichtigun...,liegt zur Besichtigung vor,NaN,NaN,NaN,6816.00,8


## 0. Before cleaning, print out how many missing values (sorted for readability) each column has:

In [6]:
# checking the missing values
(df.isnull().sum() / len(df)).sort_values(ascending = False)

parking_space          0.882199
renovated_date         0.744764
efficiency_class       0.671466
energy_requirement     0.573298
heating_costs          0.442408
utilities              0.442408
furnishing             0.417539
energy_certificate     0.409686
type_of_certificate    0.388743
energy_sources         0.353403
heating_type           0.325916
year_construction      0.000000
region                 0.000000
condition              0.000000
rooms                  0.000000
rent                   0.000000
n_missing              0.000000
space                  0.000000
cold_rent              0.000000
security_deposit       0.000000
deposit_cleaned        0.000000
property_condition     0.000000
url                    0.000000
dtype: float64

## 1. Clean: `heating_type`

Please explain why you choose your strategy for filling the missing values.

**Strategy**
- Replace each missing heating_type value with the predominant heating type of the apartment's century (year of construction in century)

**Why?**
- Since heating_type is a category data, it will be impossible to use the Mean or Median
- Using the Mode might also not reflect reality. Heating types changes with technological advancement 
- If heating types changes according to technological advancement, it is important to take the year of construction into account while filling the missing value i.e. if gas heating is the latest heating technology in 1800, most houses in this era will most likely have a gas heating system

In [7]:
# Taking the Mode will mean replacing the missing value with Central Heating
df.heating_type.value_counts()

Central Heating        201
District Heating       149
Underfloor Heating     127
Floor Heating           24
Gas Heating             10
CHP Plants               2
Wood Pellet Heating      2
Name: heating_type, dtype: int64

In [8]:
# Categorizing into 3 era: before 1900, 1900 - 2000 and after 2000
df_year_1900 =df[df['year_construction'] <=1900]
df_year_1900['heating_type'].value_counts() 

Central Heating       32
District Heating      10
Floor Heating          7
Gas Heating            3
Underfloor Heating     2
Name: heating_type, dtype: int64

In [9]:
# Since Central heating the most prodominant in this era, I will replace the missing value in this era with Central Heating
df_year_1900['heating_type'] = df_year_1900['heating_type'].replace(np.nan, 'Central Heating')
df_year_1900['heating_type'].isnull().sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0

In [10]:
df_year_2000 = df[(df['year_construction'] > 1900) & (df['year_construction'] <= 2000)]
df_year_2000['heating_type'].value_counts()

Central Heating       86
District Heating      38
Floor Heating         13
Underfloor Heating     6
Gas Heating            5
Name: heating_type, dtype: int64

In [11]:
# I replace the missing value in this era with Central heating too
df_year_2000['heating_type'] = df_year_2000['heating_type'].replace(np.nan, 'Central Heating')
df_year_2000['heating_type'].isnull().sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0

In [12]:
df_year_above_2000 =df[df['year_construction'] > 2000]
df_year_above_2000['heating_type'].value_counts()

Underfloor Heating     119
District Heating       101
Central Heating         83
Floor Heating            4
Gas Heating              2
CHP Plants               2
Wood Pellet Heating      2
Name: heating_type, dtype: int64

In [13]:
# I replace the missing value in this era with Underfloor Heating
df_year_above_2000['heating_type'] = df_year_above_2000['heating_type'].replace(np.nan, 'Underfloor Heating')
df_year_above_2000['heating_type'].isnull().sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0

In [14]:
# Bringing back dataset from each era into one combined data set
combined = pd.concat([df_year_1900, df_year_2000, df_year_above_2000], ignore_index=True)
combined['heating_type'].isnull().sum()

0

In [15]:
combined.sample(10)

,url,region,condition,rooms,rent,year_construction,space,cold_rent,utilities,heating_costs,security_deposit,parking_space,renovated_date,property_condition,furnishing,heating_type,energy_sources,energy_certificate,type_of_certificate,energy_requirement,efficiency_class,deposit_cleaned,n_missing
745,https://www.immobilienscout24.de/expose/109139762,Mitte,well_kept,2.0,1350.00,2014,68.00,672.369820,NaN,NaN,nan,NaN,NaN,Unknown,NaN,Underfloor Heating,NaN,NaN,NaN,NaN,NaN,4050.00,13
110,https://www.immobilienscout24.de/expose/114734492,Tiergarten,mint_condition,3.0,2227.00,1998,105.97,1750.000000,+477€,in,nan,NaN,2017.0,Mint Condition,NaN,Central Heating,NaN,liegt vor,"Verbrauchsausweis End­energie­verbrauch 219,5 ...",NaN,G,6681.00,5
322,https://www.immobilienscout24.de/expose/99320272,Neukölln,fully_renovated,2.0,888.00,1990,68.48,442.269926,NaN,NaN,nan,NaN,NaN,Unknown,NaN,Central Heating,NaN,NaN,NaN,NaN,NaN,2664.00,13
434,https://www.immobilienscout24.de/expose/112844530,Tiergarten,no_information,4.0,2550.00,2017,121.00,2200.000000,keine,keine,2,NaN,NaN,Unknown,NaN,Underfloor Heating,NaN,NaN,NaN,NaN,NaN,5100.00,10
473,https://www.immobilienscout24.de/expose/116139053,Mitte,well_kept,3.0,1639.00,2014,96.00,1399.000000,+240€,in,4197.00,120€,NaN,Groomed,NaN,Underfloor Heating,Fernwärme Energieausweis liegt vor,liegt vor,Bedarfsausweis,65 kWh/(m²*a),NaN,4197.00,3
203,https://www.immobilienscout24.de/expose/115802387,Wilmersdorf,fully_renovated,2.0,1359.00,1903,77.00,676.852286,NaN,NaN,nan,NaN,NaN,Unknown,NaN,Central Heating,NaN,NaN,NaN,NaN,NaN,4077.00,13
512,https://www.immobilienscout24.de/expose/114141720,Prenzlauer,mint_condition,3.0,2435.00,2015,122.98,1212.755935,NaN,NaN,nan,NaN,NaN,Unknown,NaN,Underfloor Heating,NaN,NaN,NaN,NaN,NaN,7305.00,13
360,https://www.immobilienscout24.de/expose/115695194,Prenzlauer,first_time_use,3.0,2812.00,2019,100.70,2490.000000,+322€,in,7470.00,NaN,NaN,New,Luxus,Central Heating,Gas,NaN,Verbrauchsausweis Energie­ver­brauchs­kennwert...,NaN,NaN,7470.00,5
630,https://www.immobilienscout24.de/expose/115358240,Tiergarten,first_time_use,4.0,2499.59,2019,122.83,2131.100000,"+245,66€","+122,83€",4263,NaN,2019.0,New,Gehobene Qualität,District Heating,Fernwärme Energieausweis liegt vor,liegt vor,Bedarfsausweis,"66,2 kWh/(m²*a)",B,4263.00,1
449,https://www.immobilienscout24.de/expose/114030902,Mitte,mint_condition,4.0,4192.63,2015,191.13,2088.146577,NaN,NaN,nan,NaN,NaN,Unknown,NaN,Underfloor Heating,NaN,NaN,NaN,NaN,NaN,12577.89,13


## 2. Clean: `renovated_date`

Please explain why you choose your strategy for filling the missing values.

**Strategy**
- Replace each missing renovated_date value with the median of renovated date

**Why?**
- Since most apartments were renovated during this year, I decided to use the median
- Revovated date is an important factor considered when renting an apartment. Hence, this column is important for further analysis and could not be dropped

In [16]:
combined['renovated_date'].value_counts()

2019.0    121
2020.0     26
2016.0     13
2017.0     12
2018.0     11
2015.0      4
1995.0      2
2011.0      2
2014.0      1
2012.0      1
2010.0      1
2006.0      1
Name: renovated_date, dtype: int64

In [17]:
combined['condition'].value_counts()

first_time_use                        270
mint_condition                        143
no_information                        111
well_kept                              96
refurbished                            40
first_time_use_after_refurbishment     36
fully_renovated                        35
modernized                             29
need_of_renovation                      4
Name: condition, dtype: int64

In [18]:
combined['renovated_date'].mean()

2018.1692307692308

In [19]:
combined['renovated_date'].median()

2019.0

In [20]:
combined['renovated_date'] = combined['renovated_date'].fillna(2019)
combined['renovated_date'].isnull().sum()

0

In [21]:
(combined.isnull().sum() / len(combined)).sort_values(ascending = False)

parking_space          0.882199
efficiency_class       0.671466
energy_requirement     0.573298
heating_costs          0.442408
utilities              0.442408
furnishing             0.417539
energy_certificate     0.409686
type_of_certificate    0.388743
energy_sources         0.353403
space                  0.000000
region                 0.000000
condition              0.000000
rooms                  0.000000
rent                   0.000000
year_construction      0.000000
n_missing              0.000000
cold_rent              0.000000
security_deposit       0.000000
deposit_cleaned        0.000000
renovated_date         0.000000
property_condition     0.000000
heating_type           0.000000
url                    0.000000
dtype: float64

## 3. Clean: `parking_space`

Please explain why you choose your strategy for filling the missing values.

**Strategy**
- Missing parking_space values are not filled

**Why?**
- Most house in the region where values are missing in reality do not have parking space 

In [22]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764 entries, 0 to 763
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   url                  764 non-null    object 
 1   region               764 non-null    object 
 2   condition            764 non-null    object 
 3   rooms                764 non-null    float64
 4   rent                 764 non-null    float64
 5   year_construction    764 non-null    int64  
 6   space                764 non-null    float64
 7   cold_rent            764 non-null    float64
 8   utilities            426 non-null    object 
 9   heating_costs        426 non-null    object 
 10  security_deposit     764 non-null    object 
 11  parking_space        90 non-null     object 
 12  renovated_date       764 non-null    float64
 13  property_condition   764 non-null    object 
 14  furnishing           445 non-null    object 
 15  heating_type         764 non-null    obj

In [23]:
## code here
combined['parking_space'].unique()

array([nan, '70€', '0€', '100€', '150€', '60€', '145€', '200€', '90€',
       '149€', '75€', '120€', '130€', '350€', '119€', '140€', '220€',
       '110€', '166,6€', '180€', '250€', '280€'], dtype=object)

In [24]:
combined['parking_space'] = combined['parking_space'].str.replace('€', '')
combined['parking_space'].unique()

array([nan, '70', '0', '100', '150', '60', '145', '200', '90', '149',
       '75', '120', '130', '350', '119', '140', '220', '110', '166,6',
       '180', '250', '280'], dtype=object)

In [25]:
combined['parking_space'] = combined['parking_space'].str.replace(',', '.')
combined['parking_space'].unique()

array([nan, '70', '0', '100', '150', '60', '145', '200', '90', '149',
       '75', '120', '130', '350', '119', '140', '220', '110', '166.6',
       '180', '250', '280'], dtype=object)

In [27]:
combined['parking_space'] = combined['parking_space'].astype(float)
combined['parking_space'].mean()

139.66222222222223

In [28]:
combined['parking_space'].mode()

0    120.0
dtype: float64

In [29]:
combined['parking_space'].median()

120.0

In [30]:
combined.groupby('region')[['parking_space']].mean()

,parking_space
region,
Charlottenburg,120.900000
Friedrichsfelde,0.000000
Kreuzberg,125.000000
Köpenick,82.500000
Mitte,172.331707
Neukölln,98.750000
Prenzlauer,NaN
Tiergarten,116.000000
Wedding,NaN


## 4. Clean: `utilities`

Please explain why you choose your strategy for filling the missing values.

**Strategy**
- Replace each missing heating_type value with mode

**Why?**
- Mode was choosing for simplicity sake

In [32]:
combined['utilities'].unique()

array([nan, '+110€', '+40€', '+85€', '+200€', '+180€', '+65€', '+165€',
       '+258,88€', '+60€', '+145€', '+70€', '+242,10€', '+298,14€',
       '+280€', '+550€', '+760€', '+120€', '+75€', '+200,13€', '+356€',
       'keine', '+250€', '+393€', '+135€', '+310€', '+140€', '+700€',
       '+259,34€', '+100€', '+132€', '+128€', '+214€', '+254€', '+90€',
       '+294€', '+453,60€', '+150€', '+434,17€', '+330€', '+675€',
       '+151,71€', '+400€', '+270€', '+477€', '+579€', '+460€', '+265€',
       '+160€', '+97,76€', '+111,75€', '+80€', '+163€', '+80,07€', '+68€',
       '+209€', '+157,84€', '+211,75€', '+214,20€', '+274,65€',
       '+345,30€', '+312,32€', '+224,34€', '+352,80€', '+351,90€',
       '+183€', '+392,15€', '+262,16€', '+328€', '+205€', '+408,86€',
       '+130€', '+249€', '+320€', '+210€', '+295€', '+408€', '+370€',
       '+1€', '+194,42€', '+454€', '+162€', '+158€', '+217,19€',
       '+120,32€', '+195€', '+215€', '+227€', '+300€', '+147,75€',
       '+225,98€', '+266,84€

In [33]:
## removing the € sign
combined['utilities'] = combined['utilities'].str.replace('€', '').str.replace('+', '').str.replace(',', '.')
combined['utilities'].unique()

array([nan, '110', '40', '85', '200', '180', '65', '165', '258.88', '60',
       '145', '70', '242.10', '298.14', '280', '550', '760', '120', '75',
       '200.13', '356', 'keine', '250', '393', '135', '310', '140', '700',
       '259.34', '100', '132', '128', '214', '254', '90', '294', '453.60',
       '150', '434.17', '330', '675', '151.71', '400', '270', '477',
       '579', '460', '265', '160', '97.76', '111.75', '80', '163',
       '80.07', '68', '209', '157.84', '211.75', '214.20', '274.65',
       '345.30', '312.32', '224.34', '352.80', '351.90', '183', '392.15',
       '262.16', '328', '205', '408.86', '130', '249', '320', '210',
       '295', '408', '370', '1', '194.42', '454', '162', '158', '217.19',
       '120.32', '195', '215', '227', '300', '147.75', '225.98', '266.84',
       '103.44', '176.60', '201.30', '205.74', '190', '225', '178.60',
       '470', '224.62', '248.32', '173', '147', '123', '168', '63',
       '226.49', '255', '500', '750', '133.30', '225.84', '164.09'

In [34]:
combined['utilities'].mode()

0    250
dtype: object

In [35]:
combined['utilities'] = combined['utilities'].fillna(250)
combined['utilities'].isnull().sum()

0

In [39]:
# Saving the cleaned dataset into an excel file
combined.to_excel(r"/content/cleaned_berlin_housing_data.xlsx")